- 참조 

  - 텐서플로우 공식 홈페이지

# 뉴럴 스타일 트랜스퍼(Neural Style Transfer)

- 원하는 이미지를 다른 스타일의 이미지로 구성하는 기법

- 이 기법은 Neural Style Transfer로 알려져있으며, Leon A. Gatys의 논문 A Neural Algorithm of Artistic Style에 잘 기술되어 있음

- Neural style transfer은 콘텐츠 (content) 이미지와 (유명한 작가의 삽화와 같은) 스타일 참조 (style reference) 이미지를 이용하여, 콘텐츠 이미지의 콘텐츠는 유지하되 스타일 참조 이미지의 화풍으로 채색한 것 같은 새로운 이미지를 생성하는 최적화 기술

- 이 과정은 출력 이미지를 콘텐츠 이미지의 콘텐츠 통계랑(statistic)과 스타일 참조 이미지의 스타일 통계량에 맞춰 최적화시킴으로써 구현되며 통계량은 합성곱 신경망을 이용해 각각의 이미지에서 추출

- 코드 참조 : https://www.tensorflow.org/tutorials/generative/style_transfer


* Image: https://storage.googleapis.com/download.tensorflow.org/example_images/YellowLabradorLooking_new.jpg
* Style: https://storage.googleapis.com/download.tensorflow.org/example_images/Vassily_Kandinsky%2C_1913_-_Composition_7.jpg

### 입력 시각화

- 이미지를 불러오는 함수를 정의하고, 최대 이미지 크기를 512개의 픽셀로 제한

- 이미지를 출력하기 위한 간단한 함수를 정의

## TF-Hub를 통한 빠른 스타일 전이

## 콘텐츠와 스타일 표현 정의하기

- 모델의 입력층부터 시작해서, 처음 몇 개의 층은 선분이나 질감과 같은 이미지 내의 저차원적 특성에 반응

  - 반면, 네트워크가 깊어지면 최종 몇 개의 층은 바퀴나 눈과 같은 고차원적 특성들을 나타냄
  
- 이번 경우, 우리는 사전학습된 이미지 분류 네트워크인 VGG19 네트워크의 구조를 사용할 것이고,  
  이 중간층들은 이미지에서 콘텐츠와 스타일 표현을 정의하는 데 필요
  
  - 입력 이미지가 주어졌을때, 스타일 전이 알고리즘은 이 중간층들에서 콘텐츠와 스타일에 해당하는 타깃 표현들을 일치시키려고 시도할 것

- 분류층을 제외한 VGG19 모델을 불러오고, 각 층의 이름을 출력

- 이미지의 스타일과 콘텐츠를 나타내기 위한 모델의 중간층들을 선택

### 스타일과 콘텐츠를 위한 중간층

- 사전훈련된 이미지 분류 네트워크 속에 있는 중간 출력으로 어떻게 스타일과 콘텐츠 표현을 정의할 수 있을까?

  - 고수준에서 보면 (네트워크의 훈련 목적인) 이미지 분류를 수행하기 위해서는 네트워크가 반드시 이미지를 이해해야한다.
  
  - 이는 미가공 이미지를 입력으로 받아 픽셀값들을 이미지 내에 존재하는 특성(feature)들에 대한 복합적인 이해로 변환할 수 있는 내부 표현(internal representation)을 만드는 작업이 포함

  - 또한 부분적으로 왜 합성곱(convolutional) 신경망의 일반화(generalize)가 쉽게 가능한지를 나타냄  
    즉, 합성곱 신경망은 배경잡음(background noise)과 기타잡음(nuisances)에 상관없이 (고양이와 강아지와 같이)클래스 안에 있는 불변성(invariance)과 특징을 포착할 수 있음
    
  - 따라서 미가공 이미지의 입력과 분류 레이블(label)의 출력 중간 어딘가에서 모델은 복합 특성(complex feature) 추출기의 역할을 수행
  
  - 그러므로, 모델의 중간층에 접근함으로써 입력 이미지의 콘텐츠와 스타일을 추출할 수 있음

### 모델 생성

- 함수형 API를 이용해 모델을 정의하기 위해서는 모델의 입력과 출력을 지정

- `model = Model(inputs, outputs)`


### 스타일 계산

- 이미지의 콘텐츠는 중간층들의 특성 맵(feature map)의 값들로 표현

- 이미지의 스타일은 각 특성 맵의 평균과 피쳐맵들 사이의 상관관계로 설명할 수 있음

  - 이런 정보를 담고 있는 그람 행렬(Gram matrix)은 각 위치에서 특성 벡터(feature vector)끼리의 외적을 구한 후,평균값을 냄으로써 구할 수 있음 
  
  - 주어진 층에 대한 그람 행렬은 다음과 같이 계산

  ## $$ G^l_{cd} = \frac{\sum_{ij}F^l_{ijc}(x)F^l_{ijd}(x)}{IJ}$$

이 식은 `tf.linalg.einsum` 함수를 통해 쉽게 계산할 수 있습니다:

### 스타일과 컨텐츠 추출

- 스타일과 컨텐츠 텐서를 반환하는 모델

- 이미지가 입력으로 주어졌을때, 이 모델은 `style_layers`의 스타일과 `content_layers`의 콘텐츠에 대한 그람 행렬을 출력

### 경사하강법 실행

- 타깃에 대한 입력 이미지의 평균 제곱 오차를 계산한 후, 오차값들의 가중합을 구함

- 스타일과 콘텐츠의 타깃값을 지정합니다

- 최적화시킬 이미지를 담을 `tf.Variable`을 정의하고 콘텐츠 이미지로 초기화

- 이때 `tf.Variable`는 콘텐츠 이미지와 크기가 같아야 한다


- 픽셀 값이 실수이므로 [0, 1] 사이로 cliping하는 함수 정의

- 옵티마이저 생성

- 최적화를 진행하기 위해, 전체 오차를 콘텐츠와 스타일 오차의 가중합으로 정의

- `tf.GradientTape`를 사용해 이미지를 업데이트

- 구현한 알고리즘 테스트

- 더 오랫동안 최적화 진행

### 총 변위 손실

- 위의 기본 구현 방식의 한 가지 단점은 많은 고주파 아티팩(high frequency artifact)가 생겨난다는 점

- 아티팩 생성을 줄이기 위해서는 이미지의 고주파 구성 요소에 대한 레귤러리제이션(regularization) 항을 추가해야 함

  - 스타일 전이에서는 이 변형된 오차값을 총 변위 손실(total variation loss)라고 함



- 아래의 이미지들은 고주파 구성 요소가 늘어났다는 점을 보여줌

- (참고)
  - 고주파 구성 요소가 경계선 탐지기의 일종

  - 소벨 경계선 탐지기(Sobel edge detector)를 사용하면 유사한 출력 얻음

- 정규화 오차는 각 값의 절대값의 합으로 표현

- tensorflow에 이미 표준 함수가 내장되어 있음

- 따라서 오차식을 직접 구현할 필요 없음

### 다시 최적화 진행하기

- 최적화할 변수를 다시 초기화

- 최적화 수행

### 결과 저장